# This code checks if the names and the positions match.

### Functions

In [7]:
from bs4 import BeautifulSoup
import requests
from HTMLParser import HTMLParser
from datetime import datetime
import hashlib
import requests
import re

import numpy as np
import pandas as pd

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()


# In[58]:
from selenium import webdriver
import urllib2
import MySQLdb

# MAIN


### Read the files

In [8]:
NBA_dataset = pd.read_csv("NBA Dataset with RPM.csv", sep=",")
NBA_dataset.head()

,first_name,last_name,names,pos,team,GP,MPG,ORPM,DRPM,RPM,WINS
0,Giannis,Antetokounmpo,Giannis Antetokounmpo,PG,MIL,80.0,35.3,0.84,0.27,1.11,7.46
1,Jerryd,Bayless,Jerryd Bayless,SG,MIL,52.0,28.9,0.21,-2.34,-2.13,0.92
2,Michael,Carter-Williams,Michael Carter-Williams,PG,MIL,54.0,30.5,-1.73,0.37,-1.36,1.82
3,Chris,Copeland,Chris Copeland,PF,MIL,NaN,NaN,NaN,NaN,NaN,NaN
4,Jared,Cunningham,Jared Cunningham,SG,MIL,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
positions_DS = pd.read_csv("Pbp Data Positions.csv", sep=",")
positions_DS.head()

,id,p1pG,p2pG,p3pG,p4pG,p5pG,p1pO,p2pO,p3pO,p4pO,p5pO,secs,score_GSW,score_OPP,sO-sG,game
0,4637,SF,PF,C,SG,PG,SF,PF,PF,SG,PG,302,8,8,0,20160619/CLEGSW
1,4638,SF,PF,SF,SG,PG,SF,PF,PF,SG,PG,84,3,2,-1,20160619/CLEGSW
2,4639,SF,PF,SF,SG,PG,SF,SF,PF,SG,PG,185,5,6,1,20160619/CLEGSW
3,4640,C,PF,SF,SG,PG,SF,SF,PF,SG,PG,0,0,0,0,20160619/CLEGSW
4,4641,C,PF,SF,PG,PG,SF,SF,PF,SG,PG,0,0,0,0,20160619/CLEGSW


In [10]:
names_DS = pd.read_csv("Pbp Data Names.csv", sep=",")
names_DS.head()

,id,p1nG,p2nG,p3nG,p4nG,p5nG,p1nO,p2nO,p3nO,p4nO,p5nO,secs,score_GSW,score_OPP,sO-sG,game
0,4637,Harrison Barnes,Draymond Green,Festus Ezeli,Klay Thompson,Stephen Curry,LeBron James,Kevin Love,Tristan Thompson,J.R. Smith,Kyrie Irving,302,8,8,0,20160619/CLEGSW
1,4638,Harrison Barnes,Draymond Green,Andre Iguodala,Klay Thompson,Stephen Curry,LeBron James,Kevin Love,Tristan Thompson,J.R. Smith,Kyrie Irving,84,3,2,-1,20160619/CLEGSW
2,4639,Harrison Barnes,Draymond Green,Andre Iguodala,Klay Thompson,Stephen Curry,LeBron James,Richard Jefferson,Tristan Thompson,J.R. Smith,Kyrie Irving,185,5,6,1,20160619/CLEGSW
3,4640,Anderson Varejao,Draymond Green,Andre Iguodala,Klay Thompson,Stephen Curry,LeBron James,Richard Jefferson,Tristan Thompson,J.R. Smith,Kyrie Irving,0,0,0,0,20160619/CLEGSW
4,4641,Anderson Varejao,Draymond Green,Andre Iguodala,Shaun Livingston,Stephen Curry,LeBron James,Richard Jefferson,Tristan Thompson,J.R. Smith,Kyrie Irving,0,0,0,0,20160619/CLEGSW


In [11]:
# make a dict for GSW for faster searching
from datetime import datetime 
dict_GSW = {}
errors_list  = []

print str(datetime.now())

for i,row in names_DS.iterrows():
    for k in range(1,11):
        if row[k] not in dict_GSW.keys():
            
            # get the official position (NBA)
            pos = NBA_dataset[NBA_dataset['names'] == row[k]].iloc[0]['pos']
            
            dict_GSW[row[k]] = pos
        else:
            #do the checking
            pos_at_posDS = positions_DS.iloc[i][k]


            if dict_GSW[row[k]] == pos_at_posDS:
                pass
            else:
                #maybe they are traded
                pos2 = NBA_dataset[NBA_dataset['names'] == row[k]].iloc[1]['pos']
                dict_GSW[row[k]] = pos2
                if dict_GSW[row[k]] == pos_at_posDS:
                    pass
                else:
                    errors_list.append(row[k])

        
print str(datetime.now())

print "Finished checking."
if len(errors_list) == 0:
    print "No Errors."
else:
    print "There are ", len(errors_list), " errors."

2018-03-08 15:39:56.849000
2018-03-08 15:40:05.486000
Finished checking.
No Errors.
